In [ ]:
#pip install kaggle

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! kaggle competitions download -c titanic

In [ ]:
os.chdir("/kaggle/input/titanic")
train_df = pd.read_csv("train.csv")

In [ ]:
train_df.sample(5)

In [ ]:
train_df_cp = train_df.copy()

In [ ]:
train_df.describe()

In [ ]:
(train_df.Survived == 1).sum()
print("survival %", (342/891)*100)


In [ ]:
train_df.groupby(['Survived', 'Pclass']).PassengerId.count()


In [ ]:
train_df.groupby(['Survived', 'Sex']).PassengerId.count()

In [ ]:
train_df.sample(5)

In [ ]:
train_df = train_df_cp.copy()
#lets map male and female as O and 1 
map_gender = {'female': 0,
      'male': 1}
train_df['GENDER'] = train_df.Sex.map(map_gender)
embark_map = {"C": -1, "Q": 0, "S":1}
train_df['EMBARK_M'] = train_df['Embarked'].map(embark_map)

#age entries are missing so taking average age for those
train_df.loc[train_df.Age.isna(), 'Age']= train_df.Age.mean()

train_df.loc[train_df.Fare >= 33, 'Fare_class'] = 2
train_df.loc[(train_df.Fare >= 14.5)&(train_df.Fare < 33), 'Fare_class'] = 1
train_df.loc[(train_df.Fare >= 8)&(train_df.Fare < 14.5), 'Fare_class'] = 0
train_df.loc[train_df.Fare < 8, 'Fare_class'] = -1

#normalising age and fare before ML
train_df['Age'] = (train_df['Age'] - train_df['Age'].mean())/train_df['Age'].std()

train_df['Fare'] = (train_df['Fare'] - train_df['Fare'].mean())/train_df['Fare'].std()

#checking cabin info:
train_df.loc[train_df.Cabin.isna(),'Cabin']='H'

train_df['Cabin2'] = train_df.Cabin.str[0]

#train_df.Cabin2.unique() 
cabin_map = {'A':-2, 'B':-1.5, 'C':-1, 'D':-0.5 , 'E':0,'F': 0.5,  'G': 1, 'H':0, 'T':1.5}

train_df.Cabin2 = train_df.Cabin2.map(cabin_map)
train_df.sample(2)

In [ ]:
#model training

In [ ]:


#columns selected for study
train_df_parm = train_df[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'GENDER', 'EMBARK_M', 'Cabin2']]

In [ ]:
train_df_parm.EMBARK_M = train_df_parm.EMBARK_M.fillna(0)

In [ ]:
y = train_df['Survived']

In [ ]:
train_df.groupby(["Pclass","Embarked", "Survived"]).PassengerId.count()

In [ ]:
train_df.groupby(["Fare_class", "Survived"]).PassengerId.count()

In [ ]:
#lets add a catagory for cabin as well

train_df.Cabin.isna().sum()

In [ ]:
train_df_parm.describe()

#modelling with logistic regression

In [ ]:
#modelling the parameters:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

In [ ]:
train_df_parm.sample()

In [ ]:
 
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_df_parm, y, test_size=0.2, random_state=42)


In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
y_prob_score = model.predict_proba(X_test)[:, 1]

In [ ]:
model.coef_

In [ ]:
#plotting different confusion matrix to set thrreshold

for threshold in [.3,.4,.5,.6,.7,.8]:
    print(f'when threshold is:',threshold )
    #y_pred = np.array()
    y_pred = np.where(y_prob_score<threshold,0,1)
    print(confusion_matrix(y_test, y_pred))

In [ ]:
#
y_prd = np.where(y_prob_score<0.7,0,1)
accuracy = accuracy_score(y_test, y_prd)
print("Accuracy: {:.2f}%".format(accuracy * 100))
confusion_matrix(y_test, y_prd)

# Training with NN

In [ ]:
# lets use neural networks for the problems

import tensorflow as tf


nn_model = tf.keras.models.Sequential([
 
  tf.keras.layers.Dense(3, activation='relu'),
  
  tf.keras.layers.Dense(1, activation='sigmoid')
])

nn_model.compile(
              loss= 'binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
#nn_model.compile(loss= tf.keras.losses.binary_crossentropy())
nn_model.fit(X_train, y_train, epochs=5)

In [ ]:
nn_model.evaluate(X_test, y_test)

In [ ]:
#layer1, layer2 = nn_model.layers
y_prb_nn = nn_model.predict(X_test)
for threshold in [.1,.2, .3,.4,.5,.6,.7,.8, 0.9]:
    print(f'when threshold is:',threshold )
    #y_pred = np.array()
    y_pred = np.where(y_prb_nn<threshold,0,1)
    print(confusion_matrix(y_test, y_pred))


In [ ]:
test_df = pd.read_csv('test.csv')

In [ ]:
test_df2 = test_df.copy()
#lets map male and female as O and 1 
map_gender = {'female': 0,
      'male': 1}
test_df2['GENDER'] = test_df2.Sex.map(map_gender)
embark_map = {"C": -1, "Q": 0, "S":1}
test_df2['EMBARK_M'] = test_df2['Embarked'].map(embark_map)

#age entries are missing so taking average age for those
test_df2.loc[test_df2.Age.isna(), 'Age']= test_df2.Age.mean()

test_df2.loc[test_df2.Fare >= 33, 'Fare_class'] = 2
test_df2.loc[(test_df2.Fare >= 14.5)&(test_df2.Fare < 33), 'Fare_class'] = 1
test_df2.loc[(test_df2.Fare >= 8)&(test_df2.Fare < 14.5), 'Fare_class'] = 0
test_df2.loc[test_df2.Fare < 8, 'Fare_class'] = -1

#normalising age and fare before ML
test_df2['Age'] = (test_df2['Age'] - test_df2['Age'].mean())/test_df2['Age'].std()

test_df2['Fare'] = (test_df2['Fare'] - test_df2['Fare'].mean())/test_df2['Fare'].std()

#checking cabin info:
test_df2.loc[test_df2.Cabin.isna(),'Cabin']='H'

test_df2['Cabin2'] = test_df2.Cabin.str[0]

#test_df2.Cabin2.unique() 
cabin_map = {'A':-2, 'B':-1.5, 'C':-1, 'D':-0.5 , 'E':0,'F': 0.5,  'G': 1, 'H':0, 'T':1.5}

test_df2.Cabin2 = test_df2.Cabin2.map(cabin_map)
#test_df2.sample(2)

test_df2.loc[test_df2.Fare.isna(), 'Fare']= test_df2.Age.mean()

test_df_x = test_df2[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'GENDER', 'EMBARK_M', 'Cabin2']]

test_df_x.sample()

In [ ]:
 #= model.predict(test_df_x)
y_test_prob_score = model.predict_proba(test_df_x)[:, 1]

y_prdct1 = np.where(y_test_prob_score<0.7,0,1)

y_prb_nn = nn_model.predict(test_df_x)

y_prdct2 = np.where(y_prb_nn<0.7,0,1)

In [ ]:
index1 = test_df['PassengerId']

dictn = {'PassengerId' : index1, 'Survived': y_prdct1.flatten()}
out = pd.DataFrame(dictn)

out

In [ ]:
os.getcwd()
os.chdir('/kaggle/working/')

In [ ]:
import datetime as dt
today = dt.datetime.today().strftime("%Y%m%d")

In [ ]:
out.to_csv(f'titanic_lr_pred_{today}.csv', index = False)

In [ ]:
! kaggle competitions submit -c titanic -f /kaggle/working/titanic_pred.csv -m "pred_20250315"